In [293]:
#dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin
import pymongo
import time

In [294]:
browser = Browser('chrome')

In [295]:
def get_soup(url):
    browser.visit(url)
    html = browser.html    
    soup = bs(html, 'html.parser')
    return soup

In [296]:
#scraping top ten tags
soup = get_soup('http://quotes.toscrape.com/')
sidebar = soup.find_all('span', class_='tag-item')
item = sidebar[0]
tags = []

for item in sidebar:
    tag = item.a.getText()
    tags.append(tag)
    print(tag)

love
inspirational
life
humor
books
reading
friendship
friends
truth
simile


In [297]:
#scraping quotes
for x in range(10):
    soup = get_soup('http://quotes.toscrape.com/')
    quotes = soup.find_all('span', class_='text')
    for quote in quotes:
        print(quote.text)
    try:
        browser.links.find_by_partial_text('Next').click()
    except:
        print('AGAIN')

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“It is our choices, Harry, that show what we truly are, far more than our abilities.”
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
“Try not to become a man of success. Rather become a man of value.”
“It is better to be hated for what you are than to be loved for what you are not.”
“I have not failed. I've just found 10,000 ways that won't work.”
“A woman is like a tea bag; you never know how strong it is until it's in hot water.”
“A day without sunshine is like, you know, night.”
“The world as we have created it is a process of our thinking. It cannot be changed without cha

In [298]:
#scraping authors 
for x in range(1):
    soup = get_soup('http://quotes.toscrape.com/')
    authors = soup.find_all(class_='author')
    writers = []
    for author in authors:
        writers.append(author)
        print(author.text)
    try:
        browser.links.find_by_partial_text('Next').click()
    except:
        print('AGAIN')

#does not get all the author.. 
#if I set range from 1 to 10it only prints out 10 times of the same authors

Albert Einstein
J.K. Rowling
Albert Einstein
Jane Austen
Marilyn Monroe
Albert Einstein
André Gide
Thomas A. Edison
Eleanor Roosevelt
Steve Martin


In [302]:
des = get_soup('http://quotes.toscrape.com/author/Marilyn-Monroe/')
born = des.find_all('span', class_='author-born-date')
born

[<span class="author-born-date">June 01, 1926</span>]

In [300]:
#description
des = get_soup('http://quotes.toscrape.com/author/Albert-Einstein/')
description = des.find_all('div', class_='author-description')
description

[<div class="author-description">
         In 1879, Albert Einstein was born in Ulm, Germany. He completed his Ph.D. at the University of Zurich by 1909. His 1905 paper explaining the photoelectric effect, the basis of electronics, earned him the Nobel Prize in 1921. His first paper on Special Relativity Theory, also published in 1905, changed the world. After the rise of the Nazi party, Einstein made Princeton his permanent home, becoming a U.S. citizen in 1940. Einstein, a pacifist during World War I, stayed a firm proponent of social justice and responsibility. He chaired the Emergency Committee of Atomic Scientists, which organized to alert the public to the dangers of atomic warfare.At a symposium, he advised: "In their struggle for the ethical good, teachers of religion must have the stature to give up the doctrine of a personal God, that is, give up that source of fear and hope which in the past placed such vast power in the hands of priests. In their labors they will have to av